# seq2seq를 이용한 글자 레벨 기계 번역기 구현
-----------------------------
reference: [seq2seq](wikidocs.net/24996)


## 데이터 전처리

 이번 실습의 경우 주의할 점이 있다. 쌍이 되는 영어-프랑스어의 길이가 다르다. 번역을 생각해보면 당연한 것이 'I am a student'는 '나는 학생이다' 식으로 번역되기 때문에...

In [1]:
import pandas as pd
lines= pd.read_csv('.\\data\\fra.txt', names=['src', 'tar', 'CC'], sep='\t')
lines = lines.drop('CC', axis=1)
print('전체 데이터 수 : {}'.format(len(lines)))
lines = lines[:60000]
print('간이 데이터 수 : {}'.format(len(lines)))
lines.head()

전체 데이터 수 : 174481
간이 데이터 수 : 60000


,src,tar
0,Go.,Va !
1,Hi.,Salut !
2,Hi.,Salut.
3,Run!,Cours !
4,Run!,Courez !


문장의 시작과 끝을 의미하는 `<sos>`와 종료를 의미하는 `<eos>` 심볼을 넣어주어야 한다. `\t`와 `\n`를 그 대체재로 써보자.

In [2]:
lines.tar = lines.tar.apply(lambda x: '\t'+x+'\n')
lines.head()

,src,tar
0,Go.,\tVa !\n
1,Hi.,\tSalut !\n
2,Hi.,\tSalut.\n
3,Run!,\tCours !\n
4,Run!,\tCourez !\n


이제 글자 단위의 번역을 위해 글자 집합을 구축한다.

In [3]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)
        
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

79
106


In [4]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab)
print(tar_vocab)

[' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '’', '€']
['\t', '\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'Ô', 'à', 'â', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ù', 'û', 'œ', 'С', '\u2009', '\u200b', '‘', '’', '\u202f']


이제 각 글자에 인덱스를 부여한다.

In [5]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

그럼 원본 문장들에 대해 정수 인코딩을 수행한다. 인코더에는 영어가 들어가고 디코더에는 프랑스어가 대응되므로 아래와 같이 수행한다. 주의할 점은 원래 RNN 형식의 학습에는 디코더에는 `context vector`가 들어갈뿐 '입력값'이라는게 없다. 이에 대해선 아래에서 따로 설명하겠으니 일단 넘어가자.
또한 디코더의 예측값과 비교하기 위한 실제 번역된 결과가 필요한데, 이 결과에는 `<sos>`가 있을 필요가 없다. 따라서 이를 제거한 `decoder_target`도 마련해두자.

In [6]:
encoder_input = []
for line in lines.src:
    temp_X = []
    for w in line:
        temp_X.append(src_to_index[w])
    encoder_input.append(temp_X)
print(encoder_input[:5])

decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
        temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
        if t>0:
            temp_X.append(tar_to_index[w])
        t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[30, 64, 10], [31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2]]
[[1, 48, 53, 3, 4, 2], [1, 45, 53, 64, 73, 72, 3, 4, 2], [1, 45, 53, 64, 73, 72, 14, 2], [1, 29, 67, 73, 70, 71, 105, 4, 2], [1, 29, 67, 73, 70, 57, 78, 105, 4, 2]]
[[48, 53, 3, 4, 2], [45, 53, 64, 73, 72, 3, 4, 2], [45, 53, 64, 73, 72, 14, 2], [29, 67, 73, 70, 71, 105, 4, 2], [29, 67, 73, 70, 57, 78, 105, 4, 2]]


이제 패딩 작업을 수행해주자. 물론 영어-프랑스어 번역에서 입출력의 길이가 같을 필요는 없으므로 끼리끼리 패딩해주면 된다.

In [7]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

25
74


In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

패딩이 끝났으니 원-핫 인코딩을 수행해준다. 단어 단위의 번역기가 아닌 글자 단위의 번역기이기 때문에 워드 임베딩은 사용하지 않는다.

In [9]:
from tensorflow.keras.utils import to_categorical
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

## 교사 강요

현재 시점의 디코더 셀에는 입력으로 오직 이전 디코더 셀의 출력 혹은 `context vector`만 받는다고 하였다. 이제 앞서 언급하였던 `디코더의 입력값`에 대해서 설명할 차례다. 

일반적인 훈련 과정과 달리 이번에는 `이전 시점의 디코더 셀의 출력`을 `현재 시점의 디코더 셀의 입력`으로 넣어주지 않는다. 대신 `이전 시점의 실제값(정답)`을 `현재 시점의 디코더 셀의 입력값`으로 넣어준다. 그 이유는 중간에 디코더 셀의 예측이 틀렸는데 이를 다시 디코더 셀의 입력으로 사용하면 전체적으로 예측이 틀리게 되기 때문이다. 이는 훈련 시간을 느리게 만든다.

## seq2seq 기계 번역기 훈련

In [10]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [11]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [13]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=10, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 150s 3ms/step - loss: 0.4661 - val_loss: 0.5436
Epoch 2/10
48000/48000 [==============================] - 149s 3ms/step - loss: 0.3932 - val_loss: 0.4844
Epoch 3/10
48000/48000 [==============================] - 153s 3ms/step - loss: 0.3523 - val_loss: 0.4530
Epoch 4/10
48000/48000 [==============================] - 159s 3ms/step - loss: 0.3251 - val_loss: 0.4286
Epoch 5/10
48000/48000 [==============================] - 162s 3ms/step - loss: 0.3049 - val_loss: 0.4137
Epoch 6/10
48000/48000 [==============================] - 162s 3ms/step - loss: 0.2893 - val_loss: 0.4020
Epoch 7/10
48000/48000 [==============================] - 165s 3ms/step - loss: 0.2765 - val_loss: 0.3929
Epoch 8/10
48000/48000 [==============================] - 163s 3ms/step - loss: 0.2657 - val_loss: 0.3868
Epoch 9/10
48000/48000 [==============================] - 163s 3ms/step - loss: 0.2564 - val_loss: 0.3

## seq2seq 기계 번역기 동작

교사 강요 챕터에서 설명하였듯이, seq2seq는 훈련할 때와 동작할 때의 방식이 다르다. 번역 동작은 다음과 같은 단계를 거치게 된다.

1. 번역하고자 하는 입력 문장이 인코더에 들어가 은닉 상태와 셀 상태를 얻는다.
1. 상태와 `<sos>`=`\t`를 디코더로 보낸다.
1. 디코더가 `<eos>`=`\n`가 나올때까지 다음 문자를 예측하는 행동을 반복한다.

In [14]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태를 이전 상태로 사용
decoder_states = [state_h, state_c]
# 훈련 과정과 달리 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [15]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [16]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition: #stop_condition이 True가 될 때까지 루프 반복
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_char

        # <sos>에 도달하거나 최대 길이를 넘으면 중단
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태를 업데이트
        states_value = [h, c]

    return decoded_sentence

시간상의 이유로 epoch을 10만 주고 돌려보았는데, epoch이 작아서인지 성능이 그닥 나오지 않는다...

In [17]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1])
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: Run!
정답 문장: Cours !
번역기가 번역한 문장: Prends !
-----------------------------------
입력 문장: I lied.
정답 문장: J'ai menti.
번역기가 번역한 문장: J'ai payé.
-----------------------------------
입력 문장: Come in.
정답 문장: Entre.
번역기가 번역한 문장: Venez !
-----------------------------------
입력 문장: I did OK.
정답 문장: Je m'en suis bien sortie.
번역기가 번역한 문장: Je l'ai dit.
-----------------------------------
입력 문장: We're hot.
정답 문장: Nous avons chaud.
번역기가 번역한 문장: Nous sommes fouchés.
